# Kubernetes Pod Commandline Code Executor



In [ ]:
%pip -qqq install pyautogen[kubernetes]

In [ ]:
from autogen.coding.kubernetes import PodCommandLineCodeExecutor
from autogen.coding import CodeBlock

In [ ]:
from pathlib import Path
import sys
import os

kubeconfig = Path('.kube/config')

if os.environ.get('KUBECONFIG', None):
    kubeconfig = os.environ["KUBECONFIG"]
elif sys.platform == "win32":
    kubeconfig = os.environ["userprofile"] / kubeconfig
else:
    kubeconfig = os.environ["HOME"] / kubeconfig

In [ ]:
with PodCommandLineCodeExecutor(namespace="default", kube_config_file=kubeconfig) as executor:
    print(
        executor.execute_code_blocks(
            code_blocks=[
                CodeBlock(language="python", code="print('Hello, World!')"),
            ]
        )
    )

In [ ]:
with PodCommandLineCodeExecutor(
    image="node:22-alpine",
    namespace="default",
    work_dir="./app",
    timeout=10,
    kube_config_file=kubeconfig,
    execution_policies = {"javascript": True}
) as executor:
    print(
        executor.execute_code_blocks(
            code_blocks=[
                CodeBlock(language="javascript", code="console.log('Hello, World!')"),
            ]
        )
    )

In [ ]:
from kubernetes import client

pod = client.V1Pod(
    metadata=client.V1ObjectMeta(name="abcd", namespace="default", 
                                 annotations={"sidecar.istio.io/inject": "false"}),
    spec=client.V1PodSpec(
        restart_policy="Never",
        containers=[client.V1Container(
            args=["-c", "while true;do sleep 5; done"],
            command=["/bin/sh"],
            name="abcd",
            image="python:3.11-slim",
            env=[
                client.V1EnvVar(
                    name="TEST",
                    value="TEST"
                ),
                client.V1EnvVar(
                    name="POD_NAME",
                    value_from=client.V1EnvVarSource(
                        field_ref=client.V1ObjectFieldSelector(
                            field_path="metadata.name"
                        )
                    )
                )
            ]
        )],
    )
)

In [ ]:
with PodCommandLineCodeExecutor(
    pod_spec=pod,
    container_name="abcd",
    work_dir="/autogen",
    timeout=60,
    kube_config_file=kubeconfig,
) as executor:
    print(
        executor.execute_code_blocks(
            code_blocks=[
                CodeBlock(language="python", code="print('Hello, World!')"),
            ]
        )
    )
    print(
        executor.execute_code_blocks(
            code_blocks=[
                CodeBlock(code="echo $TEST $POD_NAME", language="bash"),
            ]
        )
    )
    

In [ ]:
config_list = []

In [ ]:
from autogen import ConversableAgent

# The code writer agent's system message is to instruct the LLM on how to
# use the Jupyter code executor with IPython kernel.
code_writer_system_message = """
You have been given coding capability to solve tasks using Python code.
In the following cases, suggest python code (in a python coding block) or shell script (in a sh coding block) for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill.
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action beyond executing the code you suggest. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed by the user.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned by the user.
"""
with PodCommandLineCodeExecutor(namespace="default", kube_config_file=kubeconfig) as executor:
    
    code_executor_agent = ConversableAgent(
        name="code_executor_agent",
        llm_config=False,
        code_execution_config={
            "executor": executor,
        },
        human_input_mode="NEVER",
    )

    code_writer_agent = ConversableAgent(
        "code_writer",
        system_message=code_writer_system_message,
        llm_config={"config_list": config_list},
        code_execution_config=False,  # Turn off code execution for this agent.
        max_consecutive_auto_reply=2,
        human_input_mode="NEVER",
    )

    chat_result = code_executor_agent.initiate_chat(
        code_writer_agent, message="Write Python code to calculate the 14th Fibonacci number."
    )